- refine this:
    outline process of clustering and sentiment analysis: make sure methodologically solid and produces meaningful results
    
    
- interesting: connected networks?


topic modelling playground

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

vectorizer = CountVectorizer(stop_words='english', ngram_range=(1,2))
X = vectorizer.fit_transform(more_info['combined_text'])

lda = LatentDirichletAllocation(n_components=10, random_state=0)
lda.fit(X)

# Print top terms per topic
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = f"Topic #{topic_idx+1}: "
        message += ", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

print_top_words(lda, vectorizer.get_feature_names_out(), 10)


In [19]:
# 1) Install what you need (run once in your venv)
# pip install gensim nltk pyldavis

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pyLDAvis.gensim_models
import pandas as pd

# Assume you already have more_info['combined_text']
texts_raw = more_info['combined_text'].fillna("")

# 2) Preprocess: tokenize, remove stopwords & short tokens
stop_words = set(stopwords.words("english"))
def preprocess(doc):
    tokens = simple_preprocess(doc, deacc=True)      # lowercase, strip accents/punct
    return [t for t in tokens if t not in stop_words and len(t) > 3]

texts = texts_raw.map(preprocess).tolist()

# 3) Build dictionary & corpus
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=5000)
corpus = [dictionary.doc2bow(t) for t in texts]

# 4) Train & inspect models for different topic counts
def train_and_print(k):
    model = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=k,
        random_state=42,
        passes=10,
        alpha="auto"
    )
    print(f"\n=== Top words for {k} topics ===")
    for i in range(k):
        print(f"Topic {i+1}:", [w for w,_ in model.show_topic(i, 8)])
    return model

# Your sweet spot: pick 5–10
lda7 = train_and_print(7)

# 5) Visualize the 7-topic model interactively
vis = pyLDAvis.gensim_models.prepare(lda7, corpus, dictionary)

import pyLDAvis
pyLDAvis.enable_notebook()   # tell pyLDAvis to render inline
vis                          # simply evaluating the prepared vis object will display it



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jocelynshek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



=== Top words for 7 topics ===
Topic 1: ['election', 'york', 'results', 'live', 'adams', 'mayor', 'times', 'trial']
Topic 2: ['trump', 'president', 'donald', 'harris', 'former', 'biden', 'kamala', 'campaign']
Topic 3: ['like', 'year', 'years', 'time', 'life', 'first', 'york', 'world']
Topic 4: ['israel', 'said', 'israeli', 'gaza', 'people', 'officials', 'city', 'hezbollah']
Topic 5: ['court', 'federal', 'state', 'government', 'said', 'would', 'abortion', 'chief']
Topic 6: ['today', 'puzzle', 'conversation', 'welcome', 'forum', 'polls', 'wordle', 'spelling']
Topic 7: ['people', 'company', 'could', 'health', 'million', 'hurricane', 'climate', 'year']


/Users/jocelynshek/Desktop/makerspace/panopticum_pn-analysis/venv312/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=37527) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/jocelynshek/Desktop/makerspace/panopticum_pn-analysis/venv312/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=37527) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/jocelynshek/Desktop/makerspace/panopticum_pn-analysis/venv312/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=37527) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/jocelynshek/Desktop/makerspace/panopticum_pn-analysis/venv312/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.089228 -0.089622       1        1  30.522633
1      0.145793 -0.008428       2        1  17.904957
6      0.137835 -0.059077       3        1  16.344501
3      0.113873 -0.021727       4        1  13.867213
4      0.116411  0.054170       5        1  11.548273
0     -0.217457  0.366322       6        1   5.652175
5     -0.385682 -0.241638       7        1   4.160250, topic_info=           Term          Freq         Total Category  logprob  loglift
43        trump  17176.000000  17176.000000  Default  30.0000  30.0000
39    president  12155.000000  12155.000000  Default  29.0000  29.0000
2769   election   7617.000000   7617.000000  Default  28.0000  28.0000
44         york   7882.000000   7882.000000  Default  27.0000  27.0000
1197     donald   7368.000000   7368.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
385     feeling    499.385054    656.904542   Topic7  -4.9110   2.9054
584      robert    551.969828    980.608597   Topic7  -4.8109   2.6049
41        times    801.403834   4863.109355   Topic7  -4.4381   1.3765
387        help    633.040738   2201.579718   Topic7  -4.6739   1.9332
447       first    629.076186   5996.649992   Topic7  -4.6802   0.9249

[403 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1864      5  0.999769  abortion
2198      4  0.201149   accused
2198      5  0.598640   accused
2198      6  0.199547   accused
1038      1  0.074073    across
...     ...       ...       ...
44        7  0.014335      york
531       1  0.976829     young
531       3  0.019167     young
531       4  0.004107     young
4709      2  0.997441  zelensky

[655 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 7, 4, 5, 1, 6])